# [문제3]
- law_1.docx 
- pinecone store 저장
    - index name 임의
- RetrievalQA 구현 
    - prompt: rlm/rag-prompt
    - 질문: 전세사기

In [ ]:
import os

from dotenv import load_dotenv
from langchain import hub
from langchain_community.document_loaders import Docx2txtLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pinecone import Pinecone


load_dotenv()
api_key = os.getenv('PINECONE_API_KEY')
LANGCHAIN_API_KEY = os.getenv('LANGCHAIN_API_KEY')



## 파일 읽고 분할 한번에. 
# 파인콘에 저장하고 난 뒤엔 필요 없는 코드임. 
# loader = Docx2txtLoader("law_1.docx")

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=1500,
#     chunk_overlap=200,
# )

# data_list = loader.load_and_split(text_splitter=text_splitter)

## 임베딩
embedding = OpenAIEmbeddings(
    model="text-embedding-3-large",
)

# 파인콘에 저장

pc = Pinecone(api_key=api_key)

# database = PineconeVectorStore.from_documents(
#     documents=data_list,
#     embedding=embedding,
#     index_name='law1-quiz3'
# )

# 인덱스 가져오기
# [방법1]은 index=pc.Index('law1-quiz3') 클래스로 가져오기. 

# database = PineconeVectorStore(
#     embedding=embedding,
#     index=pc.Index('law1-quiz3')
# )

# [방법2] 메서드 PineconeVectorStore.from_existing_index
# 파인콘에 인덱스가 있는지 확인 후 가져오는 것. 이게 더 안정적이긴 한데 아무거나 쓰면 됨. 

database = PineconeVectorStore.from_existing_index(
    embedding=embedding,
    index_name='law1-quiz3',
)



llm = ChatOpenAI()

prompt = hub.pull("rlm/rag-prompt")

## RetrievalQA 구현
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

chain_lcel = (
    {
        "context" : database.as_retriever() | format_docs,
        "question" : RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

chain_lcel.invoke("전세사기 피해 구제 방법에 대해 알려주세요.")

'전세사기피해자를 지원하고 주거안정을 도모하기 위해 전세사기피해자법이 시행되었습니다. 국가와 지방자치단체는 전세사기피해자 및 임차인을 위한 긴급한 주거안정을 위해 금융지원이나 지원을 제공할 수 있습니다. 또한, 전세사기피해지원위원회를 통해 관련 사항을 심의하고 결정하며, 전세사기피해자들에 대한 보호 대책을 수립해야 합니다.'